In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from nltk.tokenize import RegexpTokenizer
import math
import gensim
from gensim.models import Word2Vec
from gensim.models import FastText

In [2]:
data = pd.read_csv("train.csv",dtype=object,na_values=str).values

In [3]:
x = np.array(data[:,2:4])
y = np.array(data[:,1])

In [4]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

In [67]:
corpus_raw = []
for i in range(0,x.shape[0]):
    for sent in x[i]:
        if np.nan_to_num(sent) == 0:     # If short description is missing
            corpus_raw.append("NA")      # Necessary, otherwise while feeding training data into NN can cause mismatch
        elif np.nan_to_num(sent) != 0:
            corpus_raw.append(sent)

In [68]:
print(corpus_raw[0:5])
print(np.shape(corpus_raw))
print(type(corpus_raw[24199]))
#words = corpus_raw[0].split()
#print(words)
print(np.nan_to_num(corpus_raw[24199]))

['There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV', 'She left her husband. He killed their children. Just another day in America.', "Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song", 'Of course it has a song.', 'Hugh Grant Marries For The First Time At Age 57']
(401664,)
<class 'str'>
NA


In [69]:
# List of words
words2 = []
tokenizer = RegexpTokenizer(r'\w+')
i = 0
for sent in corpus_raw:
    #words.append(sent.split())
    #i = i + 1
    #print(i)
    for word in tokenizer.tokenize(sent):
        words2.append(word)
    
# for i in range(0,len(corpus_raw)):
#     for word in corpus_raw[i].split():
#         if word != '.':       # because we don't want to treat . as a word
#             words.append(word)
#print(type(words))
#words = np.array(words)
#print(type(words))
#print(len(words))
#print(words[0])

Vocab = set(words2)    # so that all duplicate words are removed
vocab = list(Vocab)
# vocab = []
# for j in words:
#     if j not in vocab:
#         vocab.append(j)


word2int = {}
int2word = {}
vocab_size = len(vocab)  # gives the total number of unique words

for i,word in enumerate(vocab):
    word2int[word] = i
    int2word[i] = word

In [70]:
# List of lists of words
words = []
sentences = []
tokenizer = RegexpTokenizer(r'\w+')
i = 0
for sent in corpus_raw:
    #words.append(sent.split())
    #i = i + 1
    #print(i)
    for word in tokenizer.tokenize(sent):
        words.append(word)
    sentences.append(words)
    words = []
# for i in range(0,len(corpus_raw)):
#     for word in corpus_raw[i].split():
#         if word != '.':       # because we don't want to treat . as a word
#             words.append(word)
#print(type(words))
#words = np.array(words)
#print(type(words))
#print(len(words))
#print(words[0])

#Vocab = set(words)    # so that all duplicate words are removed
#vocab = list(Vocab)
# vocab = []
# for j in words:
#     if j not in vocab:
#         vocab.append(j)


# word2int = {}
# int2word = {}
# vocab_size = len(vocab)  # gives the total number of unique words

# for i,word in enumerate(vocab):
#     word2int[word] = i
#     int2word[i] = word

In [71]:
print(len(words))
print(vocab_size)
print(words[0:100])
print(type(vocab))
#Vocab = list(vocab)
#print(vocab[0:100])
# for i in range(0,5):
#     print(i)
# for sent in corpus_raw:
#     print(type(sent))
print(len(word2int))
print(len(int2word))
#print(int2word)
print(len(sentences))
print(sentences[0:5])

0
119541
[]
<class 'list'>
119541
119541
401664
[['There', 'Were', '2', 'Mass', 'Shootings', 'In', 'Texas', 'Last', 'Week', 'But', 'Only', '1', 'On', 'TV'], ['She', 'left', 'her', 'husband', 'He', 'killed', 'their', 'children', 'Just', 'another', 'day', 'in', 'America'], ['Will', 'Smith', 'Joins', 'Diplo', 'And', 'Nicky', 'Jam', 'For', 'The', '2018', 'World', 'Cup', 's', 'Official', 'Song'], ['Of', 'course', 'it', 'has', 'a', 'song'], ['Hugh', 'Grant', 'Marries', 'For', 'The', 'First', 'Time', 'At', 'Age', '57']]


In [72]:
sentences = np.array(sentences)

In [73]:
model_word2vec = Word2Vec(sentences, size=100, window=5, min_count=0,workers=10,sg=0)
model_word2vec.train(sentences,total_examples=len(sentences),epochs=10)

(52701159, 60996040)

In [74]:
print(model_word2vec)
print(model_word2vec['killing'])
print(model_word2vec.wv.most_similar("shootings"))

Word2Vec(vocab=119541, size=100, alpha=0.025)
[-0.54685813  0.25076315  0.92616844  1.144328    2.0551136   1.5384881
 -0.62366205  1.0247073  -1.6796687  -0.8035525   0.8259742  -1.0985408
 -0.1364447   0.12807433  1.6307139   0.5654104  -0.12359933 -0.8720993
  0.37266365 -1.0864244  -2.247471    0.7853427  -0.15346748  0.55016136
  2.4135401  -0.4848523   1.8287183   0.8424177  -0.32662195 -1.6947381
  0.6835333  -0.3748434  -0.7774773  -0.4060959  -0.7597703  -0.65643007
 -0.9999848  -0.52989835 -2.3733194  -1.5348572  -1.0939271   0.33598673
 -0.5725102  -0.06677867 -0.25671172 -0.70027655  1.2763828  -1.5107672
 -1.3437667   0.7893474  -0.5292094   0.5529111  -0.10230776 -1.3250577
 -0.98092145 -1.3715086   2.222861    0.503197   -2.4020553   2.5975807
 -0.7265535   0.8000334  -0.41110936 -1.1927834   1.0251832  -1.2498313
  0.45622286  1.3940312  -0.9723995   0.6520696  -0.5341604   1.9899546
  0.3732447   0.65729994  1.6611592   1.051633   -1.6209116   1.2629538
  0.721418   -0

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


[('shooting', 0.7239995002746582), ('massacre', 0.7116231918334961), ('killings', 0.7032343149185181), ('deaths', 0.7009249925613403), ('incidents', 0.6998181343078613), ('mass', 0.6883281469345093), ('homicides', 0.6437969207763672), ('bombings', 0.6389467716217041), ('starvations', 0.6373001337051392), ('Newtown', 0.6355317831039429)]


/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [75]:
model_word2vec.save("Saved_model_word2vec")

In [76]:
model = Word2Vec.load("Saved_model_word2vec")
print(model)

Word2Vec(vocab=119541, size=100, alpha=0.025)


In [77]:
embeddings = model[model.wv.vocab]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [78]:
print(type(embeddings))
print(embeddings.shape)

<class 'numpy.ndarray'>
(119541, 100)


In [79]:
print(embeddings[2])
print(model['There'])

[ 1.7664075e-02  8.4760055e-02 -1.9771054e-02  6.4719141e-02
 -9.3166366e-02  1.5671920e-02  7.3042087e-02  1.5022810e-02
  5.3337853e-02  2.8924717e-02  3.1462985e-03 -2.5344064e-02
  5.0998092e-02 -1.3891241e-02 -8.0226980e-02 -4.7575094e-02
  3.5614710e-02  1.5196036e-02  3.1290777e-02 -2.4902407e-02
 -1.6969467e-02  5.8391348e-02  6.8663195e-02  1.6878327e-02
  4.8826538e-02 -7.7927448e-02 -1.3926639e-02  3.0664122e-02
 -6.0330216e-02  5.4414451e-02  1.9097341e-02  7.1145572e-02
 -3.5071369e-02 -2.4239130e-03  6.3200388e-03  8.5623979e-05
  3.0584808e-02  1.2126015e-02 -8.8007599e-02 -7.5291684e-03
  4.8049867e-02  3.7963040e-02 -1.3679477e-03  6.0370076e-03
 -7.0844352e-02  6.0902789e-02  1.2678598e-01 -2.5547266e-03
 -1.9077834e-02 -1.3249801e-02  5.1044647e-02 -6.7376364e-03
 -4.4337921e-02 -4.0173996e-02  5.1313840e-02 -1.3115887e-02
 -5.5694602e-02  1.5100859e-02  8.5642196e-02  1.2859231e-02
  1.5273429e-02  1.7617268e-02 -1.5281208e-02  4.9232733e-02
  2.1503551e-03  5.09890

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [80]:
#print(word2int[0])
for i in range(0,10):
    print(int2word[i])

Dependability
Practicalities
Rwandans
Penthouse
BARCELONA
Lenkiewicz
all
Lynchpin
Dearly
Pregnancies


In [86]:
#word_vectors = np.zeros([],dtype=float)
#print(words2[0:100])
print(sentences[1])

['She', 'left', 'her', 'husband', 'He', 'killed', 'their', 'children', 'Just', 'another', 'day', 'in', 'America']


In [55]:
print(model[sentences[0][1]])

[-1.5692329e+00 -3.0277991e+00 -1.0090363e+00  1.9246717e-01
 -3.8778777e+00 -1.9707137e-01  1.1885260e+00  1.8980798e+00
  3.5624394e+00 -2.7961059e+00 -1.6923198e+00 -7.3978609e-01
  3.4912262e+00  4.7844788e-01 -1.8613143e+00  2.6814097e-01
 -7.5292313e-01  8.9893353e-01  1.7882988e+00  3.5806286e+00
 -1.4572331e+00 -2.0116150e+00  9.3713030e-02 -2.3155227e+00
 -3.4935241e+00 -1.9114302e+00 -1.9495300e+00  5.1322207e+00
  1.1074586e+00 -1.8300804e-03 -4.5545396e-01  7.3448730e-01
  9.2049706e-01 -1.4450687e-01 -6.6794977e-02 -1.1179513e+00
  2.3192902e-01 -1.1375239e+00 -4.4972296e+00 -2.1239340e+00
  3.0149109e+00  9.4599456e-01  2.5476663e+00 -4.4285914e-01
  3.3666785e+00  3.4619229e+00 -2.4715867e+00  1.9553204e-01
 -1.4509677e+00 -3.5955310e-01 -2.6171827e+00 -9.1040027e-01
  1.8099520e-01  2.0810310e-03 -2.6360388e+00  3.5628765e+00
 -9.8851967e-01  2.4547322e+00 -8.1632996e-01  3.0644231e+00
 -1.5402031e+00  1.0574621e+00  3.1202373e-01 -2.4187140e+00
  2.3699937e+00  3.64100

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
